In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_curve, auc, classification_report

import tensorflow as tf

from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score




from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, precision_recall_curve, auc
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import EditedNearestNeighbours
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt

from sklearn.metrics import precision_score, recall_score, f1_score



In [2]:
df = pd.read_csv('processed_data/merged_df_net_sc.csv')


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12280 entries, 0 to 12279
Data columns (total 41 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   gross_fu                        12280 non-null  int64  
 1   gross_sc                        12280 non-null  int64  
 2   net_fu                          12280 non-null  float64
 3   net_sc                          12280 non-null  float64
 4   time_first_sc_to_first_net_fu   12280 non-null  float64
 5   electricitybill                 12280 non-null  float64
 6   heatingbill                     12280 non-null  float64
 7   grosscontractsigned             12280 non-null  float64
 8   selfipa_done                    12280 non-null  int64  
 9   zipregion_missing               12280 non-null  int64  
 10  evaluationtime_missing          12280 non-null  int64  
 11  desiredinstallationend_missing  12280 non-null  int64  
 12  electricitybill_missing         

In [18]:
# split

target_column = 'grosscontractsigned'
X = df.drop(target_column, axis=1)
y = df[target_column]

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

scaler = StandardScaler() # important for logistic regressor
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train a simple logistic regression (NO imbalance handling yet)
lr = LogisticRegression(random_state=42, max_iter=1000)
lr.fit(X_train_scaled, y_train)

# Get predictions
y_pred_proba = lr.predict_proba(X_test_scaled)[:, 1]

# Check if there's any signal
auc_score = roc_auc_score(y_test, y_pred_proba)

print("\n=== SIGNAL TEST RESULTS ===")
print(f"AUC-ROC Score: {auc_score:.3f}")
if auc_score > 0.5:
    print("✅ GOOD NEWS: Your features contain useful signal!")
    print("We can proceed with imbalance fixing strategies.")
else:
    print("❌ BAD NEWS: No signal detected. Features need work first.")




=== SIGNAL TEST RESULTS ===
AUC-ROC Score: 0.808
✅ GOOD NEWS: Your features contain useful signal!
We can proceed with imbalance fixing strategies.


In [29]:
from sklearn.metrics import f1_score, classification_report

# Calculate AUC-PR (more reliable for imbalanced data)
precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
auc_pr = auc(recall, precision)

# Random baseline for comparison
random_baseline = y_test.mean()  # 0.023

print("\n=== DETAILED SIGNAL ANALYSIS ===")
print(f"AUC-ROC: {auc_score:.3f}")
print(f"AUC-PR: {auc_pr:.3f}")
print(f"Random baseline AUC-PR: {random_baseline:.3f}")
print(f"Improvement over random: {auc_pr/random_baseline:.1f}x")

# Look at actual predictions with default 0.5 threshold
y_pred = (y_pred_proba >= 0.1).astype(int)
f1_default = f1_score(y_test, y_pred)

print(f"\nWith 0.5 threshold:")
print(f"Predicted positives: {y_pred.sum()}")
print(f"Actual positives: {y_test.sum()}")
print(f"F1 Score: {f1_default:.3f}")

# Find optimal threshold for F1 score
thresholds = np.linspace(0, 1, 100)
f1_scores = []
for thresh in thresholds:
    y_pred_thresh = (y_pred_proba >= thresh).astype(int)
    f1_scores.append(f1_score(y_test, y_pred_thresh))

optimal_threshold = thresholds[np.argmax(f1_scores)]
optimal_f1 = max(f1_scores)
y_pred_optimal = (y_pred_proba >= optimal_threshold).astype(int)

print(f"\nOptimal F1 threshold analysis:")
print(f"Best threshold: {optimal_threshold:.3f}")
print(f"Best F1 Score: {optimal_f1:.3f}")
print(f"Predicted positives at optimal: {y_pred_optimal.sum()}")

# Check top 10% of predictions
top_10_percent_threshold = np.percentile(y_pred_proba, 90)
print(f"\nTop 10% analysis:")
print(f"Threshold for top 10%: {top_10_percent_threshold:.3f}")
top_10_mask = y_pred_proba >= top_10_percent_threshold
print(f"Conversion rate in top 10%: {y_test[top_10_mask].mean():.3f}")
print(f"That's {y_test[top_10_mask].mean()/y_test.mean():.1f}x better than average!")

# Detailed classification report at optimal threshold
print(f"\nClassification Report (Optimal F1 Threshold = {optimal_threshold:.3f}):")
print(classification_report(y_test, y_pred_optimal))


=== DETAILED SIGNAL ANALYSIS ===
AUC-ROC: 0.808
AUC-PR: 0.606
Random baseline AUC-PR: 0.177
Improvement over random: 3.4x

With 0.5 threshold:
Predicted positives: 1296
Actual positives: 434.0
F1 Score: 0.415

Optimal F1 threshold analysis:
Best threshold: 0.232
Best F1 Score: 0.554
Predicted positives at optimal: 418

Top 10% analysis:
Threshold for top 10%: 0.398
Conversion rate in top 10%: 0.691
That's 3.9x better than average!

Classification Report (Optimal F1 Threshold = 0.232):
              precision    recall  f1-score   support

         0.0       0.90      0.91      0.91      2022
         1.0       0.56      0.54      0.55       434

    accuracy                           0.85      2456
   macro avg       0.73      0.73      0.73      2456
weighted avg       0.84      0.85      0.84      2456



#### First Strategy

Class Weights

In [20]:

# Test class weights with your strong signal
print("\n=== TESTING CLASS WEIGHTS ===")

# Calculate balanced class weight
pos_weight = (1 - y_train.mean()) / y_train.mean()  # About 42 for your data
print(f"Calculated positive class weight: {pos_weight:.1f}")

# Train with class weights
lr_weighted = LogisticRegression(
    class_weight={0: 1, 1: pos_weight},
    random_state=42, 
    max_iter=1000
)
lr_weighted.fit(X_train_scaled, y_train)

# Get predictions
y_pred_weighted = lr_weighted.predict_proba(X_test_scaled)[:, 1]

# Evaluate
precision_w, recall_w, _ = precision_recall_curve(y_test, y_pred_weighted)
auc_pr_weighted = auc(recall_w, precision_w)
auc_roc_weighted = roc_auc_score(y_test, y_pred_weighted)

print(f"\nClass Weights Results:")
print(f"AUC-ROC: {auc_roc_weighted:.3f}")
print(f"AUC-PR: {auc_pr_weighted:.3f}")
print(f"Improvement over baseline: {auc_pr_weighted/auc_pr:.2f}x")

# Check different thresholds
for thresh in [0.1, 0.2, 0.3]:
    pred_at_thresh = (y_pred_weighted >= thresh).astype(int)
    if pred_at_thresh.sum() > 0:
        precision_at_thresh = (y_test[pred_at_thresh == 1]).mean()
        print(f"At {thresh} threshold: {pred_at_thresh.sum()} predicted positive, {precision_at_thresh:.3f} precision")


=== TESTING CLASS WEIGHTS ===
Calculated positive class weight: 4.7

Class Weights Results:
AUC-ROC: 0.811
AUC-PR: 0.594
Improvement over baseline: 0.98x
At 0.1 threshold: 2267 predicted positive, 0.189 precision
At 0.2 threshold: 1890 predicted positive, 0.219 precision
At 0.3 threshold: 1446 predicted positive, 0.262 precision


Class weights actually made things slightly worse (AUC-PR dropped from 0.174 to 0.158)

In [23]:
# checking the difference thresholds make

# Use the original (better) model predictions
print("=== THRESHOLD OPTIMIZATION ===")
print("Using original model (no class weights)")

# Try many different thresholds
thresholds = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]
results = []

for thresh in thresholds:
    pred_at_thresh = (y_pred_proba >= thresh).astype(int)
    
    if pred_at_thresh.sum() > 0:
        precision = (y_test[pred_at_thresh == 1]).mean()
        recall = (y_test * pred_at_thresh).sum() / y_test.sum()
        
        if precision > 0 and recall > 0:
            f1 = 2 * (precision * recall) / (precision + recall)
        else:
            f1 = 0
            
        results.append({
            'threshold': thresh,
            'predicted_positive': pred_at_thresh.sum(),
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'percent_flagged': pred_at_thresh.sum() / len(y_test) * 100
        })

# Show results
for r in results:
    print(f"Threshold {r['threshold']:.2f}: {r['predicted_positive']:4d} flagged ({r['percent_flagged']:4.1f}%), "
          f"Precision {r['precision']:.3f}, Recall {r['recall']:.3f}, F1 {r['f1']:.3f}")

=== THRESHOLD OPTIMIZATION ===
Using original model (no class weights)
Threshold 0.01: 2386 flagged (97.1%), Precision 0.180, Recall 0.991, F1 0.305
Threshold 0.02: 2287 flagged (93.1%), Precision 0.188, Recall 0.991, F1 0.316
Threshold 0.03: 2190 flagged (89.2%), Precision 0.194, Recall 0.977, F1 0.323
Threshold 0.04: 2063 flagged (84.0%), Precision 0.203, Recall 0.963, F1 0.335
Threshold 0.05: 1922 flagged (78.3%), Precision 0.214, Recall 0.949, F1 0.350
Threshold 0.06: 1767 flagged (71.9%), Precision 0.226, Recall 0.922, F1 0.363
Threshold 0.07: 1645 flagged (67.0%), Precision 0.241, Recall 0.912, F1 0.381
Threshold 0.08: 1517 flagged (61.8%), Precision 0.253, Recall 0.885, F1 0.394
Threshold 0.09: 1419 flagged (57.8%), Precision 0.264, Recall 0.862, F1 0.404
Threshold 0.10: 1296 flagged (52.8%), Precision 0.277, Recall 0.827, F1 0.415


#### Second Strategy

Focal Loss

In [30]:
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

# Your existing data split
target_column = 'grosscontractsigned'
X = df.drop(target_column, axis=1)
y = df[target_column]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Scale the features for neural network
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Your Focal Loss class
class FocalLoss(tf.keras.losses.Loss):
    def __init__(self, alpha=0.25, gamma=2.0):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
    
    def call(self, y_true, y_pred):
        ce_loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)
        p_t = y_true * y_pred + (1 - y_true) * (1 - y_pred)
        alpha_t = y_true * self.alpha + (1 - y_true) * (1 - self.alpha)
        focal_loss = alpha_t * tf.pow((1 - p_t), self.gamma) * ce_loss
        return tf.reduce_mean(focal_loss)

# Simple neural network with Focal Loss
def create_model(input_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

# 1. Model with Focal Loss
print("Training model with Focal Loss...")
focal_model = create_model(X_train_scaled.shape[1])
focal_model.compile(
    optimizer='adam',
    loss=FocalLoss(alpha=0.25, gamma=2.0),
    metrics=['accuracy']
)

focal_history = focal_model.fit(
    X_train_scaled, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    verbose=0
)

# 2. Model with Binary Crossentropy
print("Training model with Binary Crossentropy...")
bce_model = create_model(X_train_scaled.shape[1])
bce_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

bce_history = bce_model.fit(
    X_train_scaled, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    verbose=0
)

# 3. Traditional methods for comparison
print("Training traditional models...")
# Logistic Regression
lr = LogisticRegression(random_state=42)
lr.fit(X_train_scaled, y_train)

# Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Make predictions
focal_pred = (focal_model.predict(X_test_scaled) > 0.5).astype(int).flatten()
bce_pred = (bce_model.predict(X_test_scaled) > 0.5).astype(int).flatten()
lr_pred = lr.predict(X_test_scaled)
rf_pred = rf.predict(X_test)

# Compare results
print("\n" + "="*50)
print("RESULTS COMPARISON")
print("="*50)

print(f"Focal Loss Accuracy:           {f1_score(y_test, focal_pred):.4f}")
print(f"Binary Crossentropy Accuracy:  {f1_score(y_test, bce_pred):.4f}")
print(f"Logistic Regression Accuracy:  {f1_score(y_test, lr_pred):.4f}")
print(f"Random Forest Accuracy:        {f1_score(y_test, rf_pred):.4f}")

print("\n" + "-"*30)
print("FOCAL LOSS DETAILED REPORT:")
print("-"*30)
print(classification_report(y_test, focal_pred))

# Show training progress
print(f"\nFocal Loss - Final Training Accuracy: {focal_history.history['f1_score'][-1]:.4f}")
print(f"Binary CE - Final Training Accuracy:  {bce_history.history['f1_score'][-1]:.4f}")
print(f"\nFocal Loss - Final Training Accuracy: {focal_history.history['precision'][-1]:.4f}")
print(f"Binary CE - Final Training Accuracy:  {bce_history.history['precision'][-1]:.4f}")

Training model with Focal Loss...


c:\Users\ElifYilmaz\OneDrive - Enpal B.V\Desktop\New folder\Project\.venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Training model with Binary Crossentropy...


c:\Users\ElifYilmaz\OneDrive - Enpal B.V\Desktop\New folder\Project\.venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Training traditional models...
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  

RESULTS COMPARISON
Focal Loss Accuracy:           0.4532
Binary Crossentropy Accuracy:  0.5008
Logistic Regression Accuracy:  0.4753
Random Forest Accuracy:        0.6119

------------------------------
FOCAL LOSS DETAILED REPORT:
------------------------------
              precision    recall  f1-score   support

         0.0       0.87      0.99      0.93      2022
         1.0       0.87      0.31      0.45       434

    accuracy                           0.87      2456
   macro avg       0.87      0.65      0.69      2456
weighted avg       0.87      0.87      0.84      2456



KeyError: 'f1_score'

In [25]:
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, classification_report, precision_score, 
                           recall_score, f1_score, average_precision_score, roc_auc_score)
import pandas as pd

# Your existing data split
target_column = 'grosscontractsigned'
X = df.drop(target_column, axis=1)
y = df[target_column]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Scale the features for neural network
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Your Focal Loss class
class FocalLoss(tf.keras.losses.Loss):
    def __init__(self, alpha=0.25, gamma=2.0):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
    
    def call(self, y_true, y_pred):
        ce_loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)
        p_t = y_true * y_pred + (1 - y_true) * (1 - y_pred)
        alpha_t = y_true * self.alpha + (1 - y_true) * (1 - self.alpha)
        focal_loss = alpha_t * tf.pow((1 - p_t), self.gamma) * ce_loss
        return tf.reduce_mean(focal_loss)

# Simple neural network with Focal Loss
def create_model(input_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

# 1. Model with Focal Loss
print("Training model with Focal Loss...")
focal_model = create_model(X_train_scaled.shape[1])
focal_model.compile(
    optimizer='adam',
    loss=FocalLoss(alpha=0.25, gamma=2.0),
    metrics=['accuracy']
)

focal_history = focal_model.fit(
    X_train_scaled, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    verbose=0
)

# 2. Model with Binary Crossentropy
print("Training model with Binary Crossentropy...")
bce_model = create_model(X_train_scaled.shape[1])
bce_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

bce_history = bce_model.fit(
    X_train_scaled, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    verbose=0
)

# 3. Traditional methods for comparison
print("Training traditional models...")
# Logistic Regression
lr = LogisticRegression(random_state=42)
lr.fit(X_train_scaled, y_train)

# Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Make predictions
focal_pred = (focal_model.predict(X_test_scaled) > 0.5).astype(int).flatten()
bce_pred = (bce_model.predict(X_test_scaled) > 0.5).astype(int).flatten()
lr_pred = lr.predict(X_test_scaled)
rf_pred = rf.predict(X_test)

# Calculate all metrics for each model
def calculate_metrics(y_true, y_pred, y_pred_proba, model_name):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='binary')
    recall = recall_score(y_true, y_pred, average='binary')
    f1 = f1_score(y_true, y_pred, average='binary')
    auc_pr = average_precision_score(y_true, y_pred_proba)
    roc_auc = roc_auc_score(y_true, y_pred_proba)
    
    return {
        'Model': model_name,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'AUC-PR': auc_pr,
        'ROC-AUC': roc_auc
    }

# Calculate metrics for all models
results = []
results.append(calculate_metrics(y_test, focal_pred, focal_pred_proba, 'Focal Loss'))
results.append(calculate_metrics(y_test, bce_pred, bce_pred_proba, 'Binary Crossentropy'))
results.append(calculate_metrics(y_test, lr_pred, lr_pred_proba, 'Logistic Regression'))
results.append(calculate_metrics(y_test, rf_pred, rf_pred_proba, 'Random Forest'))

# Create results DataFrame
results_df = pd.DataFrame(results)

# Compare results
print("\n" + "="*80)
print("COMPREHENSIVE RESULTS COMPARISON")
print("="*80)
print(results_df.round(4).to_string(index=False))

print("\n" + "="*80)
print("RANKING BY EACH METRIC")
print("="*80)

metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC-PR', 'ROC-AUC']
for metric in metrics:
    sorted_df = results_df.sort_values(metric, ascending=False)
    print(f"\n{metric}:")
    for i, (_, row) in enumerate(sorted_df.iterrows(), 1):
        print(f"  {i}. {row['Model']}: {row[metric]:.4f}")

print("\n" + "-"*50)
print("DETAILED CLASSIFICATION REPORTS")
print("-"*50)

models = [
    ('Focal Loss', focal_pred),
    ('Binary Crossentropy', bce_pred),
    ('Logistic Regression', lr_pred),
    ('Random Forest', rf_pred)
]

for model_name, predictions in models:
    print(f"\n{model_name.upper()}:")
    print(classification_report(y_test, predictions))

# Show training progress
print(f"\nFocal Loss - Final Training Accuracy: {focal_history.history['accuracy'][-1]:.4f}")
print(f"Binary CE - Final Training Accuracy:  {bce_history.history['accuracy'][-1]:.4f}")

Training model with Focal Loss...


c:\Users\ElifYilmaz\OneDrive - Enpal B.V\Desktop\New folder\Project\.venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Training model with Binary Crossentropy...


c:\Users\ElifYilmaz\OneDrive - Enpal B.V\Desktop\New folder\Project\.venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Training traditional models...
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  


ValueError: Found input variables with inconsistent numbers of samples: [2456, 3908]

#### Third Strategy

Smoteen

In [26]:
# Configure SMOTE-ENN
smote_enn = SMOTEENN(
    smote=SMOTE(random_state=42, k_neighbors=3),  # Reduced neighbors for small minority class
    enn=EditedNearestNeighbours(n_neighbors=3),
    random_state=42
)

# Apply resampling
X_train_resampled, y_train_resampled = smote_enn.fit_resample(X_train_scaled, y_train)

print(f"Before SMOTE-ENN: {X_train_scaled.shape}, {y_train.value_counts().to_dict()}")
print(f"After SMOTE-ENN: {X_train_resampled.shape}, {pd.Series(y_train_resampled).value_counts().to_dict()}")

# Calculate the change
original_pos = y_train.sum()
resampled_pos = pd.Series(y_train_resampled).sum()
original_neg = len(y_train) - original_pos
resampled_neg = len(y_train_resampled) - resampled_pos

print(f"\nPositive samples: {original_pos} → {resampled_pos} (added {resampled_pos - original_pos})")
print(f"Negative samples: {original_neg} → {resampled_neg} (removed {original_neg - resampled_neg})")

# 5. CREATE MODEL
def create_model(input_dim, dropout_rate=0.3):
    model = Sequential([
        Dense(128, activation='relu', input_dim=input_dim),
        Dropout(dropout_rate),
        Dense(64, activation='relu'),
        Dropout(dropout_rate),
        Dense(32, activation='relu'),
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy', 'precision', 'recall', 'AUC']
    )
    
    return model

# 6. TRAIN MODEL WITH SMOTE-ENN DATA
print("\n" + "="*50)
print("TRAINING MODEL WITH SMOTE-ENN")
print("="*50)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=10,
    min_lr=1e-7,
    verbose=1
)

model_smoteenn = create_model(X_train_resampled.shape[1])

# Train on resampled data
history_smoteenn = model_smoteenn.fit(
    X_train_resampled, y_train_resampled,
    epochs=100,
    batch_size=32,
    validation_split=0.2,  # Split from resampled data
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

# 7. EVALUATE ON ORIGINAL TEST SET
print("\n" + "="*50)
print("EVALUATION ON ORIGINAL TEST SET")
print("="*50)

y_test_pred_proba = model_smoteenn.predict(X_test_scaled)
y_test_pred_proba = y_test_pred_proba.flatten()

# Calculate PR-AUC and ROC-AUC
precision_vals, recall_vals, _ = precision_recall_curve(y_test, y_test_pred_proba)
pr_auc = auc(recall_vals, precision_vals)
roc_auc = roc_auc_score(y_test, y_test_pred_proba)
random_baseline = y_test.sum() / len(y_test)

print(f"AUC-ROC: {roc_auc:.3f}")
print(f"AUC-PR: {pr_auc:.3f}")
print(f"Random baseline AUC-PR: {random_baseline:.3f}")
print(f"Improvement over random: {pr_auc/random_baseline:.1f}x")

# 8. THRESHOLD ANALYSIS (focusing on recall for your use case)
print(f"\n=== THRESHOLD ANALYSIS (RECALL-FOCUSED) ===")
thresholds = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.10, 0.15, 0.20, 0.30, 0.50]

best_recall = 0
best_threshold = 0
results = []

for threshold in thresholds:
    y_pred = (y_test_pred_proba >= threshold).astype(int)
    
    if y_pred.sum() > 0:  # Avoid division by zero
        precision = precision_score(y_test, y_pred, zero_division=0)
        recall = recall_score(y_test, y_pred, zero_division=0)
        f1 = f1_score(y_test, y_pred, zero_division=0)
        
        flagged_count = y_pred.sum()
        flagged_pct = flagged_count / len(y_test) * 100
        
        results.append({
            'threshold': threshold,
            'flagged': flagged_count,
            'flagged_pct': flagged_pct,
            'precision': precision,
            'recall': recall,
            'f1': f1
        })
        
        print(f"Threshold {threshold:.2f}: {flagged_count:4d} flagged ({flagged_pct:4.1f}%), "
              f"Precision {precision:.3f}, Recall {recall:.3f}, F1 {f1:.3f}")
        
        if recall > best_recall:
            best_recall = recall
            best_threshold = threshold

print(f"\nBest threshold for recall: {best_threshold} (Recall: {best_recall:.3f})")

# 9. COMPARE WITH YOUR PREVIOUS RESULTS
print(f"\n=== COMPARISON WITH PREVIOUS APPROACHES ===")
print("SMOTE-ENN vs Previous Results:")
print(f"Best recall achieved: {best_recall:.3f} (vs ~0.35 from class weights)")
print(f"AUC-PR: {pr_auc:.3f} (vs 0.174 from no modification)")

# 10. TOP PERFORMERS ANALYSIS
results_df = pd.DataFrame(results)
# Sort by recall (since that's your priority)
top_recall = results_df.nlargest(5, 'recall')
print(f"\nTop 5 thresholds by RECALL (your priority):")
print(top_recall[['threshold', 'flagged_pct', 'precision', 'recall', 'f1']].to_string(index=False))

# Also show balanced options
results_df['recall_precision_product'] = results_df['recall'] * results_df['precision']
balanced_options = results_df.nlargest(5, 'recall_precision_product')
print(f"\nTop 5 balanced recall-precision options:")
print(balanced_options[['threshold', 'flagged_pct', 'precision', 'recall', 'f1']].to_string(index=False))

print(f"\n" + "="*50)
print("SMOTE-ENN ANALYSIS COMPLETE")
print("="*50)
print("Key takeaways:")
print(f"1. SMOTE-ENN created {resampled_pos - original_pos} synthetic positive samples")
print(f"2. Best recall achieved: {best_recall:.3f}")
print(f"3. For your use case (capture more buyers), consider threshold around {best_threshold}")
print("4. Ready to compare with ensemble methods next")

Before SMOTE-ENN: (9824, 40), {0.0: 8087, 1.0: 1737}
After SMOTE-ENN: (14743, 40), {0.0: 8087, 1.0: 6656}

Positive samples: 1737.0 → 6656.0 (added 4919.0)
Negative samples: 8087.0 → 8087.0 (removed 0.0)

TRAINING MODEL WITH SMOTE-ENN
Epoch 1/100


c:\Users\ElifYilmaz\OneDrive - Enpal B.V\Desktop\New folder\Project\.venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


369/369 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - AUC: 0.7984 - accuracy: 0.7816 - loss: 0.4846 - precision: 0.7403 - recall: 0.4699 - val_AUC: 0.0000e+00 - val_accuracy: 0.5948 - val_loss: 0.6920 - val_precision: 1.0000 - val_recall: 0.5948 - learning_rate: 0.0010
Epoch 2/100
369/369 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - AUC: 0.8663 - accuracy: 0.8256 - loss: 0.4034 - precision: 0.8022 - recall: 0.5908 - val_AUC: 0.0000e+00 - val_accuracy: 0.5744 - val_loss: 0.7365 - val_precision: 1.0000 - val_recall: 0.5744 - learning_rate: 0.0010
Epoch 3/100
369/369 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - AUC: 0.8760 - accuracy: 0.8331 - loss: 0.3922 - precision: 0.8127 - recall: 0.6097 - val_AUC: 0.0000e+00 - val_accuracy: 0.6239 - val_loss: 0.6822 - val_precision: 1.0000 - val_recall: 0.6239 - learning_rate: 0.0010
Epoch 4/100
369/369 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - AUC: 0.8841 - accuracy: 0.8339 - loss: 0.3804 - precision: 0.7969 - recall: 0.6329 - val_AUC: 0.0000e+00 - val_accuracy: 0.6151 - val_loss: 0.65